In [6]:
import pandas as pd
import numpy as np
import yfinance as yf
import quandl as qd

In [5]:
# DOWNLOAD AAPL OHLC historical data
df_ohlc = yf.download('AAPL',start='1999-12-31', end='2010-12-31', progress=False)
df_ohlc

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1999-12-31,3.604911,3.674107,3.553571,3.671875,3.173011,40952800
2000-01-03,3.745536,4.017857,3.631696,3.997768,3.454628,133949200
2000-01-04,3.866071,3.950893,3.613839,3.660714,3.163368,128094400
2000-01-05,3.705357,3.948661,3.678571,3.714286,3.209661,194580400
2000-01-06,3.790179,3.821429,3.392857,3.392857,2.931901,191993200
...,...,...,...,...,...,...
2010-12-23,46.428570,46.450001,46.167141,46.228573,39.947918,55789300
2010-12-27,46.121429,46.491428,45.931427,46.382858,40.081257,62454000
2010-12-28,46.558571,46.665714,46.437141,46.495716,40.178780,43981000


In [8]:
# Download Consumer Price Index from Quandl
df_cpi = qd.get(dataset='RATEINF/CPI_USA', start_date='1999-12-31',  end_date='2010-12-31')
df_cpi

,Value
Date,
1999-12-31,168.300
2000-01-31,168.800
2000-02-29,169.800
2000-03-31,171.200
2000-04-30,171.300
...,...
2010-08-31,218.312
2010-09-30,218.439
2010-10-31,218.711


In [29]:
# Build the final dataset
df_ohlc.rename(columns={ 'Adj Close' : 'adj_close'}).asfreq('M')
# Pick up the adjusted close price for only end of month
df = df_ohlc[['Adj Close']].rename(columns={ 'Adj Close' : 'adj_close'}).asfreq('M')
# Merge the cpi index data into final dataset
df = df.join(df_cpi.rename(columns={'Value' : 'cpi'}),how='left')

# Find simple & log returns on adjusted close price 
df['simple_rtn'] = df.adj_close.pct_change()
df['log_rtn'] = np.log(df.adj_close/df.adj_close.shift(1))

# Find Inflation rate
df['inflation_rate'] = df.cpi.pct_change()

# Calculate inflation adjusted returns
df['real_rtn'] = ((1 + df.simple_rtn)/(1 + df.inflation_rate)) -1

df

,adj_close,cpi,simple_rtn,log_rtn,inflation_rate,real_rtn
Date,,,,,,
1999-12-31,3.173011,168.300,NaN,NaN,NaN,NaN
2000-01-31,3.201945,168.800,0.009119,0.009077,0.002971,0.006130
2000-02-29,3.537570,169.800,0.104819,0.099682,0.005924,0.098312
2000-03-31,4.191461,171.200,0.184842,0.169609,0.008245,0.175153
2000-04-30,NaN,171.300,0.000000,NaN,0.000584,-0.000584
...,...,...,...,...,...,...
2010-07-31,NaN,218.011,0.000000,NaN,0.000211,-0.000211
2010-08-31,30.010326,218.312,-0.033515,NaN,0.001381,-0.034848
2010-09-30,35.028503,218.439,0.167215,0.154621,0.000582,0.166536


In [30]:
d = yf.download('AAPL',start='2020-01-01', end='2020-08-27', progress=False)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,296.239990,300.600006,295.190002,300.350006,298.292145,33870100
2020-01-03,297.149994,300.579987,296.500000,297.429993,295.392120,36580700
2020-01-06,293.790009,299.959991,292.750000,299.799988,297.745880,29596800
2020-01-07,299.839996,300.899994,297.480011,298.390015,296.345581,27218000
2020-01-08,297.160004,304.440002,297.160004,303.190002,301.112640,33019800
...,...,...,...,...,...,...
2020-08-20,463.000000,473.570007,462.929993,473.100006,473.100006,31726800
2020-08-21,477.049988,499.470001,477.000000,497.480011,497.480011,84513700
2020-08-24,514.789978,515.140015,495.750000,503.429993,503.429993,86484400
